In [ ]:
# Check GPU configuration
!nvidia-smi

Wed Aug 21 02:44:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#STEP 0 確認有使用GPU
##記事本必須在GPU模式下運行，可以執行上方的nvidia-smi查看GPU是否有GPU。若没有，點擊左上角的 編輯>筆記本設定，把硬體加速器改成GPU。


In [ ]:
#@title STEP 1 複製儲存庫並安裝必要的函式庫
#@markdown #STEP 1
#@markdown ##複製儲存庫並安裝必要的函式庫
#@markdown ##Clone repository & Install requirements lib

!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd ./LLaMA-Factory
!pip install -r requirements.txt
!pip install bitsandbytes>=0.39.0

#用比較殘暴的方式開啟 Gradio 的分享連結
train_web_py_file_path = "./src/webui.py"
try:
  with open(train_web_py_file_path, 'r') as file:
    file_content = file.read()
  modified_content = file_content.replace("share=False", "share=True")
  with open(train_web_py_file_path, 'w') as file:
    file.write(modified_content)
except Exception as e:
    print(f'ERROR: {str(e)}')

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 16242, done.
remote: Counting objects: 100% (6978/6978), done.
remote: Compressing objects: 100% (425/425), done.
remote: Total 16242 (delta 6627), reused 6611 (delta 6550), pack-reused 9264 (from 1)
Receiving objects: 100% (16242/16242), 223.01 MiB | 14.35 MiB/s, done.
Resolving deltas: 100% (12077/12077), done.
/content/LLaMA-Factory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvid

In [ ]:
# @title STEP 1-2 安裝pyngrok {"display-mode":"both"}
# @markdown #STEP 1-2
# @markdown ## 此工具為內網穿透工具，才可能將web server暴露在公開位置
!pip install pyngrok

In [ ]:
# @title STEP 1-3 開啟7860 ports {"display-mode":"both"}
# @markdown #STEP 1-3
# @markdown  ## 指定要內網穿透的ports
# @markdown
# @markdown ## 開啟前會清除所有已執行的隧道，且設定token
from pyngrok import ngrok
# 停止所有隧道
ngrok.kill()

# 替換下面的 YOUR_AUTHTOKEN 為你從 ngrok 獲取的 Authtoken
ngrok.set_auth_token("2Wbru4ocvqZEdhYqSB6Zq6OQwno_2atqQkJP8Y9bRVPin9H1A")

# 開啟端口 7860 的 ngrok 隧道
public_url = ngrok.connect(7860)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://ce43-35-197-138-13.ngrok-free.app" -> "http://localhost:7860"


In [ ]:
# @title STEP 1-4 安裝llama-factory必要指令
# @markdown #STEP 1-4
# @markdown ## 請注意，這步驟一定要安裝，否則在訓練模型時會出錯
pip install -e .[torch,metrics]

Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.8.4.dev0-0.editable-py3-none-any.whl size=22083 sha256=7882c1da237058ee5e3976c59d8d2ecb545ff47b202a910c34a5637008d40396
  Stored in directory: /tmp/pip-ephem-wheel-cache-6zrft10q/wheels/de/aa/c5/27b5682c5592b7c0eecc3e208f176dedf6b11a61cf2a910b85
Successfully built llamafactory


In [ ]:
#@title STEP 2 開啟WebUI
#@markdown #STEP 2
#@markdown ##開啟WebUI
#@markdown ##Run the WebUI

!CUDA_VISIBLE_DEVICES=0 python src/webui.py

2024-08-21 03:10:44.807489: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 03:10:44.827142: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 03:10:44.833611: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 03:10:44.848108: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-21 03:10:46.061571: W tensorflow/comp

In [ ]:
# @title STEP 2-1 使用指令進行訓練
# @markdown #STEP 2-1
# @markdown  ## 此步驟是非必要，看個人使用習慣
! CUDA_VISIBLE_DEVICES=0 python src/train.py \
    --stage sft \
    --do_train True \
    --model_name_or_path shenzhi-wang/Llama3-8B-Chinese-Chat \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template llama3 \
    --flash_attn auto \
    --dataset_dir data \
    --dataset myDataset \
    --cutoff_len 1024 \
    --learning_rate 2e-05 \
    --num_train_epochs 1.0 \
    --max_samples 100 \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 100 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves/LLaMA3-8B-Chinese-Chat/lora/train_2024-08-20-05-04-08 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --quantization_bit 4 \
    --quantization_method bitsandbytes \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.01 \
    --eval_strategy steps \
    --eval_steps 100 \
    --per_device_eval_batch_size 1

python3: can't open file '/content/src/train.py': [Errno 2] No such file or directory


#STEP 3 上傳輸出的檔案到 HuggingFace


In [ ]:
#@title STEP 3.1 安裝HuggingFace API
#@markdown #STEP 3.1
#@markdown ##為了上傳模型至HuggingFace，需要先安裝HF的函式庫
#@markdown ##In order to upload the model to HuggingFace, we have to install the HF library first.
!pip install huggingface_hub

In [ ]:
#@title STEP 3.2 使用API Token登入
#@markdown #STEP 3.2
#@markdown ##執行本儲存格後填入 API Token(需要有 Write 權限)，然後按下登入
#@markdown ##After runing this cell, fill in the API Token (Write permission is required), and then click Login

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#@title STEP 3.3 上傳模型
#@markdown #STEP 3.3
#@markdown ##執行本儲存格後填入 API Token(需要有 Write 權限)，然後按下登入
#@markdown ##After runing this cell, fill in the API Token (Write permission is required), and then click Login
from huggingface_hub import HfApi
api = HfApi()

model_id = "username/fine-tune-model" #@param {type:"string"}
export_folder_path = "/content/LLaMA-Factory/export" #@param {type:"string"}
api.create_repo(model_id, private=True, exist_ok=True, repo_type="model")
api.upload_folder(
    folder_path=export_folder_path,
    repo_id=model_id,
)

#STEP 4 保存到Google Drive(自選)

In [ ]:
#@title STEP 4.1 連接Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title STEP 4.2 複製檔案到Google Drive
Google_Drive_Folder = "/content/drive/MyDrive/LLaMA-Factory/" #@param {type:"string"}

!cp -rf /content/LLaMA-Factory/saves {Google_Drive_Folder}